In [59]:
from magentic import prompt, ConfigDict, OpenaiChatModel
from typing import Optional, List, Dict
from dataclasses import dataclass
from datetime import date, timedelta, time
from pydantic import BaseModel

In [49]:
class Event(BaseModel):
    model_config = ConfigDict(openai_strict=True)
    name: str
    description: str
    date: date
    importance: int

In [50]:
@prompt(
    """ 
        List at least 100 events in history since 1960 that are major world events with a big economic impact.
        """,
    model=OpenaiChatModel("gpt-4o-mini-2024-07-18"),
)
def get_events() -> List[Event]: ...

events = get_events()

events

[Event(name='The Vietnam War (1955-1975)', description='A prolonged conflict between North Vietnam and South Vietnam, significant for its global economic impact through military expenditure and consequences on international relations.', date=datetime.date(1960, 1, 1), importance=5),
 Event(name='The Cuban Missile Crisis (1962)', description='A 13-day confrontation between the United States and the Soviet Union over Soviet ballistic missiles deployed in Cuba, affecting global politics and economies significantly.', date=datetime.date(1962, 10, 16), importance=5),
 Event(name='The Oil Crisis (1973)', description='An embargo by OAPEC that caused oil prices to quadruple, leading to a global recession and massive shifts in energy policies worldwide.', date=datetime.date(1973, 10, 1), importance=5),
 Event(name='The Yom Kippur War (1973)', description='Conflict between Israel and a coalition of Arab states, leading to the oil embargo and significant economic repercussions globally.', date=da

In [54]:
import pandas as pd

In [75]:
class EventWithEconomicData(Event):
    unemployment_rate: Optional[float] = None
    unemployment_rate_6m: Optional[float] = None
    unemployment_rate_12m: Optional[float] = None
    unemployment_rate_18m: Optional[float] = None
    unemployment_rate_24m: Optional[float] = None
    gdp: Optional[float] = None
    gdp_6m: Optional[float] = None
    gdp_12m: Optional[float] = None
    gdp_18m: Optional[float] = None
    gdp_24m: Optional[float] = None
    cpi: Optional[float] = None
    cpi_6m: Optional[float] = None
    cpi_12m: Optional[float] = None
    cpi_18m: Optional[float] = None
    cpi_24m: Optional[float] = None
    oil_price: Optional[float] = None
    oil_price_6m: Optional[float] = None
    oil_price_12m: Optional[float] = None
    oil_price_18m: Optional[float] = None
    oil_price_24m: Optional[float] = None


def add_economic_data(
    events: List[Event],
    unemployment_file: str,
    gdp_file: str,
    cpi_file: str,
    oil_price_file: str,
) -> List[EventWithEconomicData]:
    # Read CSV files
    unemployment_df = pd.read_csv(
        unemployment_file, parse_dates=["DATE"], index_col="DATE"
    )
    gdp_df = pd.read_csv(gdp_file, parse_dates=["DATE"], index_col="DATE")
    cpi_df = pd.read_csv(cpi_file, parse_dates=["DATE"], index_col="DATE")
    oil_price_df = pd.read_csv(oil_price_file, parse_dates=["DATE"], index_col="DATE")

    # Function to get economic data for a given date
    def get_data(date, df, column_name):
        timestamp = pd.Timestamp(date)
        return df[column_name].asof(timestamp)

    # Process events
    return [
        EventWithEconomicData(
            **event.model_dump(),
            unemployment_rate=get_data(event.date, unemployment_df, "UNRATE"),
            unemployment_rate_6m=get_data(
                event.date + timedelta(days=180), unemployment_df, "UNRATE"
            ),
            unemployment_rate_12m=get_data(
                event.date + timedelta(days=365), unemployment_df, "UNRATE"
            ),
            unemployment_rate_18m=get_data(
                event.date + timedelta(days=545), unemployment_df, "UNRATE"
            ),
            unemployment_rate_24m=get_data(
                event.date + timedelta(days=730), unemployment_df, "UNRATE"
            ),
            gdp=get_data(event.date, gdp_df, "GDPC1"),
            gdp_6m=get_data(event.date + timedelta(days=180), gdp_df, "GDPC1"),
            gdp_12m=get_data(event.date + timedelta(days=365), gdp_df, "GDPC1"),
            gdp_18m=get_data(event.date + timedelta(days=545), gdp_df, "GDPC1"),
            gdp_24m=get_data(event.date + timedelta(days=730), gdp_df, "GDPC1"),
            cpi=get_data(event.date, cpi_df, "CORESTICKM159SFRBATL"),
            cpi_6m=get_data(event.date + timedelta(days=180), cpi_df, "CORESTICKM159SFRBATL"),
            cpi_12m=get_data(event.date + timedelta(days=365), cpi_df, "CORESTICKM159SFRBATL"),
            cpi_18m=get_data(event.date + timedelta(days=545), cpi_df, "CORESTICKM159SFRBATL"),
            cpi_24m=get_data(event.date + timedelta(days=730), cpi_df, "CORESTICKM159SFRBATL"),
            oil_price=get_data(event.date, oil_price_df, "WTISPLC"),
            oil_price_6m=get_data(
                event.date + timedelta(days=180), oil_price_df, "WTISPLC"
            ),
            oil_price_12m=get_data(
                event.date + timedelta(days=365), oil_price_df, "WTISPLC"
            ),
            oil_price_18m=get_data(
                event.date + timedelta(days=545), oil_price_df, "WTISPLC"
            ),
            oil_price_24m=get_data(
                event.date + timedelta(days=730), oil_price_df, "WTISPLC"
            )
        )
        for event in events
    ]

In [76]:
events_with_economic_data = add_economic_data(events=events, unemployment_file="UNRATE.csv", gdp_file="GDPC1.csv", cpi_file="CORESTICKM159SFRBATL.csv", oil_price_file="WTISPLC.csv")

In [78]:
import csv
from typing import List


def save_events_to_csv(events: List[EventWithEconomicData], filename: str):
    # Define the field names
    fieldnames = [
        "name",
        "description",
        "date",
        "importance",
        "unemployment_rate",
        "unemployment_rate_6m",
        "unemployment_rate_12m",
        "unemployment_rate_18m",
        "unemployment_rate_24m",
        "gdp",
        "gdp_6m",
        "gdp_12m",
        "gdp_18m",
        "gdp_24m",
        "cpi",
        "cpi_6m",
        "cpi_12m",
        "cpi_18m",
        "cpi_24m",
        "oil_price",
        "oil_price_6m",
        "oil_price_12m",
        "oil_price_18m",
        "oil_price_24m",
    ]

    with open(filename, "w", newline="") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Write the header
        writer.writeheader()

        # Write each event as a row
        for event in events:
            writer.writerow(
                {
                    "name": event.name,
                    "description": event.description,
                    "date": event.date,
                    "importance": event.importance,
                    "unemployment_rate": event.unemployment_rate,
                    "unemployment_rate_6m": event.unemployment_rate_6m,
                    "unemployment_rate_12m": event.unemployment_rate_12m,
                    "unemployment_rate_18m": event.unemployment_rate_18m,
                    "unemployment_rate_24m": event.unemployment_rate_24m,
                    "gdp": event.gdp,
                    "gdp_6m": event.gdp_6m,
                    "gdp_12m": event.gdp_12m,
                    "gdp_18m": event.gdp_18m,
                    "gdp_24m": event.gdp_24m,
                    "cpi": event.cpi,
                    "cpi_6m": event.cpi_6m,
                    "cpi_12m": event.cpi_12m,
                    "cpi_18m": event.cpi_18m,
                    "cpi_24m": event.cpi_24m,
                    "oil_price": event.oil_price,
                    "oil_price_6m": event.oil_price_6m,
                    "oil_price_12m": event.oil_price_12m,
                    "oil_price_18m": event.oil_price_18m,
                    "oil_price_24m": event.oil_price_24m,
                }
            )


# Example usage:
# events = [EventWithEconomicData(...), EventWithEconomicData(...), ...]
# save_events_to_csv(events, 'economic_data.csv')

In [79]:
save_events_to_csv(events_with_economic_data, "events_with_economic_data.csv")